In [2]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Libraries imported.


## Paris Map - Current residence and venues in neighborhood for comparison to future Manhattan place

In [3]:
# My home latitude and longtitude:
neighborhood_latitude=48.8287779
neighborhood_longitude=2.3204412

### Dial FourSquare to find venues around current residence in Paris

In [4]:
CLIENT_ID = 'RMIIV1E3XURB3KHIH2OWQU4TW2BGS0RV04WH1TXVA53XQGUO' # your Foursquare ID
CLIENT_SECRET = 'PORQIUSNK0N2PHSGAQ5QU25IP2GPEHNT1EGLHZ5BAIJKJLWH' # your Foursquare Secret
VERSION = '20200716' # Foursquare API version

In [5]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RMIIV1E3XURB3KHIH2OWQU4TW2BGS0RV04WH1TXVA53XQGUO&client_secret=PORQIUSNK0N2PHSGAQ5QU25IP2GPEHNT1EGLHZ5BAIJKJLWH&v=20200716&ll=48.8287779,2.3204412&radius=500&limit=100'

In [6]:
# results display is hidden for report simplification 
results = requests.get(url).json()

In [7]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']
Parisnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Parisnearby_venues =Parisnearby_venues.loc[:, filtered_columns]
# filter the category for each row
Parisnearby_venues['venue.categories'] = Parisnearby_venues.apply(get_category_type, axis=1)
# clean columns
Parisnearby_venues.columns = [col.split(".")[-1] for col in Parisnearby_venues.columns]

Parisnearby_venues.shape

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


(33, 4)

In [9]:
# Venues near current Paris residence place
Parisnearby_venues.head(10)

,name,categories,lat,lng
0,Les Petits Plats,French Restaurant,48.828652,2.322138
1,Le Jeroboam,Modern European Restaurant,48.829527,2.317492
2,Chez Kim Ly,Vietnamese Restaurant,48.828537,2.316201
3,La Régalade,French Restaurant,48.825026,2.320244
4,Le 14 Juillet,French Restaurant,48.828450,2.316137
5,Au Moulin Vert,French Restaurant,48.829773,2.323024
6,Le Petit Baigneur,French Restaurant,48.831712,2.324259
7,Le Severo,French Restaurant,48.832074,2.324704
8,Citadines Didot Montparnasse,Hotel,48.827892,2.315296
9,Place Victor et Hélène Basch,Plaza,48.828033,2.326719


### Map of Paris - 14 ème arrondissement residence place with venues in Neighborhood - for reference

In [10]:
latitude=48.8287779
longitude=2.3204412
# create map of Paris 14 ème place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label in zip(Parisnearby_venues['lat'], Parisnearby_venues['lng'], Parisnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg

## MANHATTAN NEIGHBORHOODS - DATA AND MAPPING

### Cluster neighborhood data was produced with Foursquare during course lab work. A csv file was produced containing the neighborhoods around the 40 Boroughs. Now, the csv file is just read for convenience and consolidation of report.

In [13]:
# Read csv file with clustered neighborhoods with geodata
manhattan_data  = pd.read_csv('manhattan_data.csv') 
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [14]:
# Manhattan Borough neighborhoods - data with top 10 clustered venues
manhattan_merged = pd.read_csv('manhattan_merged.csv')
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Gym,Coffee Shop,Seafood Restaurant,Tennis Stadium,Big Box Store,Supplement Shop,Steakhouse,Miscellaneous Shop,Shopping Mall,Yoga Studio
1,Manhattan,Chinatown,40.715618,-73.994279,4,Chinese Restaurant,Cocktail Bar,Bakery,Hotpot Restaurant,Dessert Shop,Spa,Bubble Tea Shop,American Restaurant,Optical Shop,Vietnamese Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Mobile Phone Shop,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Latin American Restaurant,Tapas Restaurant,Spanish Restaurant,Italian Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Lounge,Café,Restaurant,Park,Bakery,Frozen Yogurt Shop,Caribbean Restaurant,Chinese Restaurant,Spanish Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Bakery,Cocktail Bar,Sandwich Place,Chinese Restaurant,School


### Map of Manhattan neighborhoods with top 10 clustered venues
### popups allow us to identify each neighborhood and the cluster of venues around it in order to proceed to examine in more detail in the next cell

In [15]:
# create map of Manhattan using latitude and longitude values from Nominatim
latitude= 40.7308619
longitude= -73.9871558 

kclusters=5
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
  # add markers for rental places to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)    
    
       
map_clusters

## Examine a paticular Cluster - print venues

### After examining several cluster data , I concluded that cluster # 3 resembles closer to the Paris place, therefore providing guidance as to where to look for the future apartment .
### Assigning a value to 'kk' to explore a given cluster.

In [27]:
## kk is the cluster number to explore
kk = 4
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == kk, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,Hotpot Restaurant,Dessert Shop,Spa,Bubble Tea Shop,American Restaurant,Optical Shop,Vietnamese Restaurant
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Latin American Restaurant,Tapas Restaurant,Spanish Restaurant,Italian Restaurant
3,Inwood,Mexican Restaurant,Lounge,Café,Restaurant,Park,Bakery,Frozen Yogurt Shop,Caribbean Restaurant,Chinese Restaurant,Spanish Restaurant
4,Hamilton Heights,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Bakery,Cocktail Bar,Sandwich Place,Chinese Restaurant,School
5,Manhattanville,Coffee Shop,Deli / Bodega,Liquor Store,Seafood Restaurant,Mexican Restaurant,Italian Restaurant,Falafel Restaurant,Bike Trail,Sushi Restaurant,Lounge
6,Central Harlem,African Restaurant,Cosmetics Shop,Art Gallery,Chinese Restaurant,Seafood Restaurant,Bar,French Restaurant,American Restaurant,Food Truck,Caribbean Restaurant
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Sandwich Place,Liquor Store,Cocktail Bar,Donut Shop,Beer Bar
11,Roosevelt Island,Park,Bridge,Gym,Restaurant,Greek Restaurant,Coffee Shop,Soccer Field,Liquor Store,Sandwich Place,Scenic Lookout
20,Lower East Side,Chinese Restaurant,Art Gallery,Coffee Shop,Bakery,Pizza Place,Japanese Restaurant,Ramen Restaurant,Cocktail Bar,Yoga Studio,Clothing Store
25,Manhattan Valley,Bar,Yoga Studio,Thai Restaurant,Coffee Shop,Pizza Place,Mexican Restaurant,Dog Run,Bubble Tea Shop,Café,Caribbean Restaurant


## Results
### Let's consolidate all the required inforamtion to make the apartment selection in one map

In [22]:
# create map of Manhattan using latitude and longitude values from Nominatim
latitude= 40.7308619
longitude= -73.9871558

map_mh_one = folium.Map(location=[latitude, longitude], zoom_start=13.3)


# set color scheme for the clusters
kclusters=5
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_mh_one)

    # Adds tool to the top right
from folium.plugins import MeasureControl
map_mh_one.add_child(MeasureControl())

# Measurement ruler icon tool to measure distances in map
from folium.plugins import FloatImage
url = ('https://media.licdn.com/mpr/mpr/shrinknp_100_100/AAEAAQAAAAAAAAlgAAAAJGE3OTA4YTdlLTkzZjUtNDFjYy1iZThlLWQ5OTNkYzlhNzM4OQ.jpg')
FloatImage(url, bottom=5, left=85).add_to(map_mh_one)

map_mh_one

### After examining, I have chosen the locations that meets the requirements which will assess to make a choice.
### Financial District Neighborhood, Cluster 4: Financial District having Gym, Hotels and Restuarents similar to Paris residence.
### Similar Venue - Financil District in Cluster 4

In [26]:
## kk is the cluster number to explore
kk = 4
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == kk, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,Hotpot Restaurant,Dessert Shop,Spa,Bubble Tea Shop,American Restaurant,Optical Shop,Vietnamese Restaurant
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Latin American Restaurant,Tapas Restaurant,Spanish Restaurant,Italian Restaurant
3,Inwood,Mexican Restaurant,Lounge,Café,Restaurant,Park,Bakery,Frozen Yogurt Shop,Caribbean Restaurant,Chinese Restaurant,Spanish Restaurant
4,Hamilton Heights,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Bakery,Cocktail Bar,Sandwich Place,Chinese Restaurant,School
5,Manhattanville,Coffee Shop,Deli / Bodega,Liquor Store,Seafood Restaurant,Mexican Restaurant,Italian Restaurant,Falafel Restaurant,Bike Trail,Sushi Restaurant,Lounge
6,Central Harlem,African Restaurant,Cosmetics Shop,Art Gallery,Chinese Restaurant,Seafood Restaurant,Bar,French Restaurant,American Restaurant,Food Truck,Caribbean Restaurant
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Latin American Restaurant,Deli / Bodega,Sandwich Place,Liquor Store,Cocktail Bar,Donut Shop,Beer Bar
11,Roosevelt Island,Park,Bridge,Gym,Restaurant,Greek Restaurant,Coffee Shop,Soccer Field,Liquor Store,Sandwich Place,Scenic Lookout
20,Lower East Side,Chinese Restaurant,Art Gallery,Coffee Shop,Bakery,Pizza Place,Japanese Restaurant,Ramen Restaurant,Cocktail Bar,Yoga Studio,Clothing Store
25,Manhattan Valley,Bar,Yoga Studio,Thai Restaurant,Coffee Shop,Pizza Place,Mexican Restaurant,Dog Run,Bubble Tea Shop,Café,Caribbean Restaurant


## Venue Selection
### Using the "one map" above, I was able to explore all possibilities since the popups provide the information needed for a good decision.
### Financial District having Gyms, Hotels and Restuarents similar toParis residence is my preferabe choice for a future residence.
### Based on current Paris venue, I feel that Cluster 4 type of venues is a closer resemblance to my current place. That means that Financial District is a better choice since it has Hotels and Restuarent and is worth the conveniences it provides.